In [1]:
import pandas as pd
import numpy as np

epidemic_dir = "epidemic/"


C:\Users\crystallwi.xiaujing\AppData\Local\Continuum\anaconda3\envs\env01\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\crystallwi.xiaujing\AppData\Local\Continuum\anaconda3\envs\env01\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\crystallwi.xiaujing\AppData\Local\Continuum\anaconda3\envs\env01\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


### IDEAS:
* Of the daily Covid cases, how many are hospitalised?
* Of the daily Covid cases, how many are in the PKRC?
* Time loss from work for PKRC quarantine.

## Cases

In [7]:
cases = pd.read_csv(epidemic_dir +'cases_malaysia.csv')
cases.head()
cases['date'] = pd.to_datetime(cases['date'])
print ('Cases---')
cases.describe()

## Data wrangling
cases = cases.replace(np.nan, 0, inplace=True)

Cases---


## Hospital & ICU

In [3]:
hospital = pd.read_csv(epidemic_dir + "hospital.csv")
icu = pd.read_csv(epidemic_dir+ "icu.csv")

print ('---------Hospitals Dataset --------')
hospital.state.value_counts()
print ('\n', hospital.columns)
print (hospital.describe(include='all'))
hospital['date'] = pd.to_datetime(hospital['date'])

print ('\n ------- Available hospital bed ----')
state_hospital_bed = hospital.groupby('state')['beds'].max()
print (state_hospital_bed)

print ('\n ---------- ICU Dataset ----------')
icu['date'] = pd.to_datetime(icu['date'])
print (icu.describe(include='all'))
print (icu.dtypes)
print (icu.shape)
icu_by_state = icu.state.value_counts()
print (icu_by_state)

---------Hospitals Dataset --------

 Index(['date', 'state', 'beds', 'beds_noncrit', 'admitted_pui',
       'admitted_covid', 'admitted_total', 'discharged_pui',
       'discharged_covid', 'discharged_total', 'hosp_covid', 'hosp_pui',
       'hosp_noncovid'],
      dtype='object')
              date   state         beds  beds_noncrit  admitted_pui  \
count         7363    7363  7363.000000   7363.000000   7363.000000   
unique         486      16          NaN           NaN           NaN   
top     2021-05-25  Perlis          NaN           NaN           NaN   
freq            16     486          NaN           NaN           NaN   
mean           NaN     NaN  1884.747929   1815.651229     12.485128   
std            NaN     NaN  1203.180905   1158.975342     32.099075   
min            NaN     NaN   122.000000     95.000000      0.000000   
25%            NaN     NaN  1158.000000   1124.000000      0.000000   
50%            NaN     NaN  1454.000000   1381.000000      2.000000   
75%    

C:\Users\crystallwi.xiaujing\AppData\Local\Continuum\anaconda3\envs\env01\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  app.launch_new_instance()


In [4]:
def get_min_max_date(df):
    min_date = df.date.min().date()
    max_date = df.date.max().date()
    print (f'Min date: {min_date} ')
    print (f'Max date: {max_date} ')

print ("Hospital ---")
get_min_max_date(hospital)

print("ICU ---")
get_min_max_date(icu)


Hospital ---
Min date: 2020-03-24 
Max date: 2021-07-22 
ICU ---
Min date: 2020-03-24 
Max date: 2021-07-22 


## PKRC

PKRC describes the movement of patients in and out of Covid-19 Quarantine Centre, with capacity and utilisation each day. 

In [5]:
pkrc = pd.read_csv(epidemic_dir+ "pkrc.csv")
pkrc['date'] = pd.to_datetime(pkrc['date'])

print ("PKRC---")
get_min_max_date(pkrc)
print(pkrc.describe())
pkrc.tail()


PKRC---
Min date: 2020-03-28 
Max date: 2021-07-22 
               beds  admitted_pui  admitted_covid  admitted_total  \
count   5471.000000   5471.000000     5471.000000     5471.000000   
mean    1594.633157      1.760190       71.383294       73.143484   
std     2448.078388      5.689006      174.185404      175.157427   
min       26.000000      0.000000        0.000000        0.000000   
25%      117.000000      0.000000        0.000000        0.000000   
50%      600.000000      0.000000        7.000000        9.000000   
75%     1660.000000      1.000000       61.000000       63.000000   
max    11212.000000    190.000000     2040.000000     2042.000000   

       discharge_pui  discharge_covid  discharge_total   pkrc_covid  \
count    5471.000000      5471.000000      5471.000000  5471.000000   
mean        1.418936        69.547066        70.966003   487.449095   
std         6.283023       174.635119       175.447634   944.178289   
min         0.000000         0.000000     

,date,state,beds,admitted_pui,admitted_covid,admitted_total,discharge_pui,discharge_covid,discharge_total,pkrc_covid,pkrc_pui,pkrc_noncovid
5466,2021-07-22,Sabah,6711,15,362,377,33,282,315,2891,66,0
5467,2021-07-22,Sarawak,8003,13,89,102,2,92,94,2352,194,0
5468,2021-07-22,Selangor,9639,1,259,260,0,323,323,2212,5,1
5469,2021-07-22,Terengganu,1508,23,135,158,8,82,90,863,88,0
5470,2021-07-22,W.P. Labuan,441,3,35,38,3,4,7,152,43,0


In [6]:
pkrc_bed_by_state = pd.DataFrame(pkrc.groupby(['state'])['beds'].max())
pkrc_bed_by_state.sort_values('beds', ascending=False)

,beds
state,
Selangor,11212
Sabah,9546
Sarawak,9202
Johor,4383
Negeri Sembilan,3197
Pahang,2908
Kelantan,2763
Terengganu,2742
Penang,2031


## Visualisation